In [ ]:
#imports
import os
import json
import numpy as np
import pandas as pd
import pyarrow as pa
from pandas.io.json import json_normalize

In [ ]:
#to sample part of the data set, change dates to a list of dates or use the commented code to take a mostly-random sample 
#running this block with no changes will load the entire dataset

all_dates =  ['newNONTORbenigndata'] + os.listdir('/data/data')[1:-3]
dates = all_dates
'''
dates = []
sample_size = 250
for i in range(sample_size):
    randv = np.random.rand(0,len(all_dates))
    try:
        dates.append(all_dates.pop(randv))
    except: 
        dates.append(all_dates.pop())
'''
df_lst = []
unsafe_lst = []
unsafe_req_ids=[]
safe_lst = []
safe_req_ids=[]
for d in dates:
    try:
        date = d
        enrichment_path = '/data/data/'+date+'/enrichment'
        alexa_path = enrichment_path+'/alexa'
        files = os.listdir(alexa_path)
        print(date)
        for i in range(len(files)):
            with open(alexa_path+'/'+files[i]) as d:
                try:
                    r=json.load(d)

                    try:
                        temp_df=json_normalize(r['data'])
                        labels = []
                        if len(files[i]) <= 10 :
                            labels += [0]*len(temp_df)
                        else :
                            labels += [1]*len(temp_df)
                        temp_df['unsafe'] = labels
                        df_lst.append(temp_df)
                       
                    except:
                        pass
                except:
                    print(date)
    except:
        print('something broke')
            



In [ ]:
#bring all frames together and drop an empty column
try:
    df = pd.concat(df_lst, axis = 0, join = 'outer', ignore_index = True )
except:
    print(df_lst)
    df = df_lst[0]
df = df.drop(columns=['alexa_info'])
df

In [ ]:
#renaming columns to shorter meaningful names; columns I plan to drop are not renamed
cols = df.columns
cols = np.array(cols)
df = df.rename(columns = {cols[0] : 'requestID', cols[1]:'status_code', cols[2]:'res_status_xmlns', cols[3] : 'adult_content', 
                         cols[4] : 'url_text', cols[5] : 'url_type', cols[6] : 'language', cols[7] : 'lang_encoding', 
                          cols[8]:'lang_locale', cols[9]:'links_in_count', cols[11]:'owned_domains', 
                          cols[14] : 'description',cols[15]:'online_since', cols[16]:'title',cols[17] : 'med_load_time',
                         cols[18] : 'load_speed_percentile', cols[19] : 'category_data', cols[20]:'category_path',
                          cols[21]: 'category', cols[24]:'subdoms', cols[28]:'contr_subdom'
                          , cols[36]:'rank',cols[38]:'rank_by_country', cols[44]:'use_stats', cols[48]:'months'})#, 
                          #cols[66]: 'alexa_rank',cols[67]:'indicator',cols[68]:'success'})
df_full = df
df.head(2).T

#cols[20],cols[21],cols[22],cols[25?],cols[27?,28?],cols[30,31,33,36,37,38,39,40,41,42,43,44,45,46,47,] can be dropped

In [ ]:
#dropping unneeded columns
cols = df.columns
vals = [0,1,3,4,5,7,8,9,11,14,15,16,17,18,19,20,21,24,28,36,38,44,67,68]
df2 = df
for i in range(70):
    if i in vals:
        print(str(i)+' passed') #just shows progress during runtime
    else:
        df2 = df2.drop(columns=[df.columns[i]])

#df2 = df[[cols[0],cols[1],cols[2],cols[3],cols[4],cols[5],cols[7],cols[8],cols[9],cols[11],
 #         cols[14],cols[15],cols[16],cols[18],cols[19],cols[20],cols[21],cols[24],cols[28],cols[36],cols[38],cols[44],cols[66],cols[67],cols[68]]]
df2 = df2.drop(columns=['alexa_info.html.body.center.h1','alexa_info.html.head.title'])
df2.head()

In [ ]:
#saves simple elements of dataframe, but tosses a fair amount
df=df2
df['links_in_count'] = pd.to_numeric(df['links_in_count'])
df['load_speed_percentile'] = pd.to_numeric(df['load_speed_percentile'])
df['med_load_time'] = pd.to_numeric(df['med_load_time'])
df['rank'] = pd.to_numeric(df['rank'])
df['online_since'] = pd.to_datetime(df['online_since'],errors = 'coerce',yearfirst=True)
#df['alexa_rank'] = pd.to_numeric(df['alexa_rank'],errors='coerce')
df.drop(columns = ['owned_domains','category_data','subdoms','contr_subdom','rank_by_country','use_stats']).to_parquet('updated_main_df2.gzip',compression='gzip')

In [ ]:
#create binary df
df_bin = df2.copy()
for c in df_bin.columns[1:]:
    df_bin[c] = df_bin[c].notna()
df_bin = df_bin.replace({True:1,False:0})
df_bin.to_parquet('ft_presence_df.gzip',compression='gzip')
df_bin

## Usage Data

In [ ]:
#let's look at the usage statistics data
temp = df[['indicator','use_stats']]

 #each item in use data is an array with 4 dictionaries that have more dicts inside them
temp = temp.dropna().reset_index(drop=True)
use_data = temp['use_stats']
temp.head()

In [ ]:
#this function can definitley be written much better than it is here
def parse_use_data(n,ind):

    udf = pd.DataFrame(use_data[n])
    
    lstpv = []
    lstpv_delta = []
    lstpvpu = []
    lstpvpu_delta = []
    
    lstpv_rank = []
    lstpv_rank_delta = []
    
    lstra = []
    lstra_delta = []
    lstre = []
    lstre_delta = []
    
    lstre_rank = []
    lstre_rank_delta = []
    
    lsttime = []
    lstind = []
    
    for i in range(len(udf['aws:PageViews'])):
        d = udf['aws:PageViews'][i].get('aws:PerMillion')
        v = d.get('aws:Value')
        if v is None: 
            lstpv.append(np.nan)
            lstpv_delta.append(np.nan) 
        else:
            lstpv.append((float(v.replace(',',''))))
            v = d.get('aws:Delta')
            if v is None or v ==' ':
                lstra_delta.append(np.nan)
            else:
                try:
                    lstpv_delta.append(float(v.replace('%','').replace('+',''))) # need to switch from percentage string to float
                except:
                    lstpv_delta.append(np.nan)
                    
        e = udf['aws:Rank'][i]
        v = e.get('aws:Value')
        if v is None: 
            lstra.append(np.nan)
            lstra_delta.append(np.nan) 
        else:
            lstra.append(float(v.replace(',','')))
            v = e.get('aws:Delta')
            if v is None:
                lstra_delta.append(np.nan)
            else:
                try:
                    lstra_delta.append(float(v)) # not a percentage 
                except:
                    lstra_delta.append(np.nan)
                    
        f = udf['aws:Reach'][i].get('aws:PerMillion')
        v = f.get('aws:Value')
        if v is None: 
            lstre.append(np.nan)
            lstre_delta.append(np.nan) 
        else:
            lstre.append(float(v.replace(',','')))
            v = f.get('aws:Delta')
            if v is None:
                lstre_delta.append(np.nan) # need to switch from percentage string
            else:
                try:
                    lstre_delta.append(float(v.replace('%','').replace('+','')))
                except:
                    lstre_delta.append(np.nan)
        #standardizing a month as 30 days for time_range
        g = udf['aws:TimeRange'][i]
        gm = g.get('aws:Months')
        gd = g.get('aws:Days')
        val = 0
        if gm != None:
            val+= int(gm)*30
        if gd != None: 
            val+= int(gd)
        lsttime.append(val)
        lstind.append(ind)
        
        h = udf['aws:PageViews'][i].get('aws:PerUser')
        v = h.get('aws:Value')
        if v is None: 
            lstpvpu.append(np.nan)
            lstpvpu_delta.append(np.nan)
        else:
            lstpvpu.append(float(v.replace(',','')))
            v = h.get('aws:Delta')
            if v is None:
                lstpvpu_delta.append(np.nan) # need to switch from percentage string
            else:
                try:
                    lstpvpu_delta.append(float(v.replace('%','').replace('+','')))
                except:
                    lstpvpu_delta.append(np.nan)
        
        k = udf['aws:PageViews'][i].get('aws:Rank')
        v = k.get('aws:Value')
        if v is None: 
            lstpv_rank.append(np.nan)
            lstpv_rank_delta.append(np.nan)
        else:
            lstpv_rank.append(float(v.replace(',','')))
            v = k.get('aws:Delta')
            if v is None:
                lstpv_rank_delta.append(np.nan) # need to switch from percentage string
            else:
                try:
                    lstpv_rank_delta.append(float(v.replace('+','')))
                except:
                    lstpv_rank_delta.append(np.nan)
                    
        l = udf['aws:PageViews'][i].get('aws:PerUser')
        v = l.get('aws:Value')
        if v is None: 
            lstre_rank.append(np.nan)
            lstre_rank_delta.append(np.nan)
        else:
            lstre_rank.append(float(v.replace(',','')))
            v = l.get('aws:Delta')
            if v is None:
                lstre_rank_delta.append(np.nan) # need to switch from percentage string
            else:
                try:
                    lstre_rank_delta.append(float(v.replace('+','')))
                except:
                    lstre_rank_delta.append(np.nan)
        
    udf['indicator'] = lstind
    udf['pg_views_per_mil'] = lstpv
    udf['pg_views_per_mil_%change'] = lstpv_delta
    udf['pg_views_per_user'] = lstpvpu
    udf['pg_views_per_user_%change'] = lstpvpu_delta
    udf['pg_views_rank'] = lstpv_rank
    udf['pg_views_rank_change'] = lstpv_rank_delta
    udf['rank'] = lstra
    udf['rank_change'] = lstra_delta
    udf['reach_per_mil'] = lstre
    udf['reach_per_mil_%change'] = lstre_delta
    udf['reach_rank'] = lstre_rank
    udf['reach_rank_change'] = lstre_rank_delta
    udf['time_range_days'] = lsttime
    udf = udf[['indicator','pg_views_per_mil','pg_views_per_mil_%change', 'pg_views_per_user','pg_views_per_user_%change', 
               'pg_views_rank','pg_views_rank_change','rank','rank_change','reach_per_mil','reach_per_mil_%change',
               'reach_rank','reach_rank_change','time_range_days']]
    return udf

In [ ]:
res_dfs = []
for i in range(len(temp['indicator'])):
    res_dfs.append(parse_use_data(i,temp['indicator'][i]))
    #print(i) #uncomment to print out index so progress is visible
usage_data_df = pd.concat(res_dfs, axis = 0, join = 'outer', ignore_index = True)
#usage_data_df = usage_data_df.merge(labels)
usage_data_df.to_parquet('usage_df.gzip',compression = 'gzip')
usage_data_df.head(15)

## Country Rank Data

In [ ]:
temp = df[['indicator','rank_by_country']]
temp = temp.dropna().reset_index(drop=True)

country_data = temp['rank_by_country']
country_data[0]
cdf = pd.DataFrame(country_data[0])
cdf.head()

In [ ]:
temp = df[['indicator','rank_by_country']]
temp = temp.dropna().reset_index(drop=True)

country_data = temp['rank_by_country']
def parse_country_data(n,ind):
    cdf = pd.DataFrame(country_data[n])
    
    lst_pv = []
    lst_us = []
    lst_ind = []
    for i in range(len(cdf['Code'])):
        v = cdf['aws:Contribution'][i].get('aws:PageViews')
        if v is None : 
            lst_pv.append(np.nan)
        else:
            lst_pv.append(float((cdf['aws:Contribution'][i].get('aws:PageViews')).replace('%',''))/100)
            
        v = cdf['aws:Contribution'][i].get('aws:Users')
        if v is None:
            lst_us.append(np.nan)
        else:
            lst_us.append(float((cdf['aws:Contribution'][i].get('aws:Users')).replace('%',''))/100)
        lst_ind.append(ind)
    cdf['indicator'] = lst_ind
    cdf['pg_view_contribution'] = lst_pv
    cdf['user_contribution'] = lst_us
    cdf = cdf.rename(columns={'Code':'country_code', 'aws:Contribution': 'contribution', 'aws:Rank':'rank'})
    cdf = cdf[['indicator','country_code','rank', 'pg_view_contribution','user_contribution']]
    return cdf

res_dfs = []
for i in range(len(temp['indicator'])):
    res_dfs.append(parse_country_data(i,temp['indicator'][i]))
    print(i)
country_rank_df = pd.concat(res_dfs, axis = 0, join = 'outer', ignore_index = True)
country_rank_df = country_rank_df.merge(labels)
country_rank_df.to_parquet('country_rank_df.gzip',compression = 'gzip')
country_rank_df.head()

## Owned Domains Data

In [ ]:
ddf = df[['indicator','owned_domains']].dropna().reset_index(drop=True)
lst_ind = []
lst_dom = []
lst_title = [] 

for i in range(len(ddf['indicator'])):
    d = ddf['owned_domains'][i]
    for x in d:
        lst_ind.append(ddf['indicator'][i])
        lst_dom.append(x.get('aws:Domain'))
        lst_title.append(x.get('aws:Title'))
domain_df = pd.DataFrame({'indicator':lst_ind, 'domain_name':lst_dom,'domain_title':lst_title})
domain_df = domain_df[['indicator','domain_name','domain_title']]
domain_df.to_parquet('owned_domain_df.gzip',compression='gzip')
domain_df

## Contributing Subdomain Data

In [ ]:
sddf = df[['indicator','contr_subdom']].dropna().reset_index(drop=True)
lst_ind = []
lst_url = []
lst_pvpu = []
lst_pvperc = []
lst_reperc = []
lst_time = []

for i in range(len(sddf['indicator'])):
    ind = sddf['indicator'][i]
    for x in sddf['contr_subdom'][i]:
        lst_ind.append(ind)
        lst_url.append(x.get('aws:DataUrl'))
        lst_pvpu.append(float(x.get('aws:PageViews').get('aws:PerUser')))
        lst_pvperc.append(float(x.get('aws:PageViews').get('aws:Percentage').replace('%','')))
        lst_reperc.append(float(x.get('aws:Reach').get('aws:Percentage').replace('%','')))
        days = 0
        try:
            days += int(x.get('aws:TimeRange').get('aws:Months'))*30
        except:
            days += 0
        try:
            days += int(x.get('aws:TimeRange').get('aws:Days'))
        except: 
            days += 0
        lst_time.append(days)
contr_subdom_df = pd.DataFrame({'indicator':lst_ind, 'data_url':lst_url, 'page_views_per_user':lst_pvpu, 
                                'page_views_%':lst_pvperc, 'reach_%':lst_reperc, 'time_range_days': lst_time})
contr_subdom_df.to_parquet('contributing_subdomain_df.gzip',compression='gzip')
contr_subdom_df
        

## Category Data

In [ ]:
catdf = df[['indicator','category_data']].dropna().reset_index(drop=True)
lst_ind = []
lst_title=[]
lst_path = []
for i in range(len(catdf['indicator'])):
    ind = catdf['indicator'][i]
    for x in catdf['category_data'][i]:
        lst_ind.append(ind)
        lst_title.append(x.get('aws:Title'))
        lst_path.append(x.get('aws:AbsolutePath'))
category_df = pd.DataFrame({'indicator':lst_ind, 'path':lst_path,'title':lst_title})
category_df.to_parquet('category_df.gzip',compression = 'gzip')
category_df